In [2]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [12]:
data_dirs = ['../data/results_D100'] # , '../data/results_D25']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [13]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[1].split('/')[0]
    experiment = fname_pieces[2]
    n_mca = int(fname_pieces[3])
    target = fname_pieces[-3]
    classif = fname_pieces[-2]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Target": target,
        "Experiment": experiment,
        "MCA Simulations": n_mca,
        "Classifier": classif,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and n_mca != 20:
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [14]:
df_results

,Target,Experiment,MCA Simulations,Classifier,Dataset,rs,aggregation,acc,f1,test_acc,test_f1,test_mean_acc,test_mean_f1,n_models
0,vo2max,mca,10,LR,D100,42,mega,0.506844,0.531293,0.529954,0.457447,0.510599,0.468564,5
1,vo2max,mca,2,LR,D100,42,median,0.578333,0.612100,0.550000,0.571429,0.510000,0.507287,5
2,sex,mca,20,RF,D100,42,consensus,0.626667,0.737628,0.500000,0.500000,0.530000,0.598751,5
3,age,mca,20,SVM,D100,42,ref,0.744167,0.814978,0.550000,0.709677,0.550000,0.671459,5
4,bmi,mca,10,SVM,D100,42,none,0.597767,0.473959,0.684000,0.680335,0.740600,0.703281,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,cholesterol,mca,20,RF,D100,42,ref,0.781667,0.000000,0.800000,0.000000,0.790000,0.000000,5
461,vo2max,mca,20,SVM,D100,42,consensus,0.500000,0.609569,0.500000,0.375000,0.480000,0.600222,5
462,age,mca,10,SVM,D100,42,consensus,0.705833,0.777416,0.700000,0.823529,0.570000,0.692924,5
463,age,mca,2,LR,D100,42,median,0.731667,0.800960,0.600000,0.733333,0.560000,0.696664,5


In [15]:
df_results.to_csv("../data/aggregation_results_if_indiv.csv", index=False)